# Descomposición CUR

### Carga de Librerías (Python 3.7)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from numpy.linalg import svd
from numpy.linalg import linalg
from time import time
import random

### Importar archivo con calificaciones (ratings)

In [2]:
# Conjunto de datos con un millón de calificaciones (cifra redondeada)
ratings = pd.read_csv(r'ratings_1m.csv')

# Separar conjunto 75% entrenamiento y 25% validación, se usa train_test_split de sklearn
train, test = train_test_split(ratings,test_size=0.25)

### Mapeo de Usuarios y de Películas

In [3]:
# El mapeo lo que hace es tener un id consecutivo y único de usuarios y de películas
# La tabla original de películas no tiene id consecutivos

lista_pelic = ratings['movieId'].unique()
num_pelic = len(lista_pelic)
mapeo_pelic = {}
for n, m in enumerate(lista_pelic):
    mapeo_pelic[m] = n
lista_usuarios = ratings['userId'].unique()
num_usuarios = len(lista_usuarios)
mapeo_usuarios = {}
for n, m in enumerate(lista_usuarios):
    mapeo_usuarios[m] = n

### Creación de Matriz de usuarios y películas

In [4]:
matriz_uspl = np.zeros([num_usuarios, num_pelic])
for index, row in ratings.iterrows():
    matriz_uspl[mapeo_usuarios[row['userId']], mapeo_pelic[row['movieId']]] = int(row['rating'])

# Se guarda una copia como Data Frame
matriz_df = pd.DataFrame(matriz_uspl)    

Rango de la matriz de usuarios y peliculas

In [5]:
np.linalg.matrix_rank(matriz_uspl)

3663

$(k \, log \, k ) / \epsilon^2$

In [6]:
import math
k = 800
epsilon = 1/3
print((k * math.log10(k)), k/epsilon)

2322.471989593555 2400.0


In [7]:
print(num_usuarios, num_pelic)
matriz_df.head()

6040 3706


,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,5.0,3.0,3.0,4.0,5.0,3.0,5.0,5.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Descomposición CUR

In [8]:
# Se contabiliza el tiempo de procesamiento para la Descomposición CUR
tiempo_inicio = time()
random.seed(5829)

# Cálculo de los promedios de las calificaciones de cada usuario
prom_usr = matriz_uspl.sum(axis=1)
aux = Counter(matriz_uspl.nonzero()[0])
for i in range(num_usuarios):
    if i in aux.keys():
        prom_usr[i] = prom_usr[i]/aux[i]
    else:
        prom_usr[i] = 0

# Cálculo de los promedios de las calificaciones por película
prom_pelic = matriz_uspl.T.sum(axis=1)
aux = Counter(matriz_uspl.T.nonzero()[0])
for i in range(num_pelic):
    if i in aux.keys():
        prom_pelic[i] = prom_pelic[i]/aux[i]
    else:
        prom_pelic[i] = 0

# Se calculan las probabilidades de seleccionar columnas y renglones
total_norm =  np.linalg.norm(matriz_uspl)  # Calcula la norma de la matriz de usuarios y películas
col_norm =  np.linalg.norm(matriz_uspl,axis = 0) #calcula la norma del vector columna
row_norm =  np.linalg.norm(matriz_uspl,axis = 1) #calcula la norma del vector renglón

for i in range(num_pelic):
    col_norm[i] = (col_norm[i]/total_norm)**2
    
for i in range(num_usuarios):
    row_norm[i] = (row_norm[i]/total_norm)**2

# Con las probabilidades calculadas se seleccionan las columnas aleatoriamente y sin duplicar

# Para el conjunto de datos de 100,000 calificaciones se reducirá a 90 columnas
# Se hicieron pruebas con números más grandes, la SVD no convergía
# Para el conjunto de datos de 1,000,000 calificaciones se reducirá a 800 columnas

c=800
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),c)
    
for i in range(c):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1

# Con las probabilidades calculadas se seleccionan los renglones aleatoriamente y sin duplicar
# Para el conjunto de datos de 100,000 calificaciones se reducirá a 90 renglones
    
r=800
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),r)

for i in range(r):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR = np.dot(np.dot(C, U), R.T)

CUR_df = pd.DataFrame(CUR)
# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR.shape[0]):
    for j in range(CUR.shape[1]):
        if CUR[i,j] > 5:
            CUR[i,j] = 5
        elif CUR[i,j] < 0:
            CUR[i,j] = 0
tiempo_fin = time()

Número de columnas y renglones seleccionadas

In [9]:
print(W.shape[0], W.shape[1])

800 800


In [10]:
print("Dimensión Matriz Usuarios y Películas:", matriz_uspl.shape[0], matriz_uspl.shape[1])
print("Dimensión C:",C.shape[0],C.shape[1])
print("Dimensión U:",U.shape[0],U.shape[1])
print("Dimensión R:",R.shape[1],R.shape[0])

Dimensión Matriz Usuarios y Películas: 6040 3706
Dimensión C: 6040 800
Dimensión U: 800 800
Dimensión R: 800 3706


#### Función para cálculo de Error Cuadrático Medio (RMSE)

In [11]:
# Error Cuadrático Medio (RMSE)
def ECM(pred,original):
    N = pred.shape[0] # filas
    M = pred.shape[1] # columnas
    cur_sum = np.sum(np.square(pred-original))
    return np.sqrt(cur_sum/(N*M))

#### Función para cálculo de Coeficiente de Correlación de Spearman (rho)

En estadística, el coeficiente de correlación de Spearman, $\rho$  es una medida de correlación
El estadístico $\rho$ viene dado por la expresión:
$$ \rho = 1 - \frac{6 \Sigma D^2}{N(N^2-1)} $$
donde D es la diferencia entre los correspondientes estadísticos de orden x-y. N es el número de parejas de datos

La correlación de Spearman evalúa relaciones monótonas (sean lineales o no). 

En matemáticas una función entre conjuntos ordenados se dice monótona si conserva el orden dado

Intiuitivamente, la correlación de Spearman entre dos observaciones será alta (correlación cercana a 1) cuando las observaciones tengan un rango similar

In [12]:
# Coeficiente de correlación de Spearman (rho)
def rho(pred, original):
    num = np.sum(np.square(pred - original))
    n = pred.shape[0]*pred.shape[1]
    den = n*(n*n-1)
    return 1 - (6*num)/den

### Resultados Descomposición CUR

In [13]:
print("Error Cuadrático Medio", ECM(CUR, matriz_uspl))
print("Coeficiente de correlación de Spearman", rho(CUR, matriz_uspl))
print("Tiempo de procesamiento en segundos", tiempo_fin - tiempo_inicio)

Error Cuadrático Medio 3.196050201150994
Coeficiente de correlación de Spearman 0.9999999999998777
Tiempo de procesamiento en segundos 22.008479118347168


In [14]:
print(CUR.shape[0], CUR.shape[1])
print(matriz_uspl.shape[0],matriz_uspl.shape[1])

6040 3706
6040 3706


In [15]:
CUR_df = pd.DataFrame(CUR)

In [16]:
CUR_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,...,0.000158,0.000788,0.003200,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,5.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,...,0.003882,0.019408,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.003486,0.017430,0.011553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,0.0,...,0.000439,0.002193,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,5.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Descomposición en Valores Singulares (SVD)

#### Cálculo de Eigen Valores y Eigen Vectores

In [17]:
tiempo_inicial_SVD = time()
m1 = np.dot(matriz_uspl, matriz_uspl.T)
eigenValues, eigenVectors = linalg.eig(m1)
eigenValues = eigenValues.real
eigenVectors = eigenVectors.real
eigen_map = dict()
for i in range(len(eigenValues)):
    eigenValues[i] = round(eigenValues[i], 4)
for i in range(len(eigenValues)):
    if eigenValues[i] != 0:
        eigen_map[eigenValues[i]] = eigenVectors[:, i]

eigenValues = sorted(eigen_map.keys(), reverse=True)
U = np.zeros_like(eigenVectors)
for i in range(len(eigenValues)):
    U[:,i] = eigen_map[eigenValues[i]]
U = U[:,:len(eigenValues)]

#### Cálculo de Sigma

In [18]:
Sigma = np.diag([i**0.5 for i in eigenValues])

#### Cálculo de V

In [19]:
m2 = np.dot(matriz_uspl.T, matriz_uspl)
eigenValues, eigenVectors = linalg.eig(m2)
eigenValues = eigenValues.real
eigenVectors = eigenVectors.real
for i in range(len(eigenValues)):
    eigenValues[i] = round(eigenValues[i], 4)
eigen_map = dict()
for i in range(len(eigenValues)):
    if eigenValues[i] != 0:
        eigen_map[eigenValues[i]] = eigenVectors[:, i]
eigenValues = sorted(eigen_map.keys(), reverse=True)
V = np.zeros_like(eigenVectors)
for i in range(len(eigenValues)):
    V[:,i] = eigen_map[eigenValues[i]]
V = V[:,:len(eigenValues)]
V = V.T

t_aux_1 = time()

#### Cálculo SVD con 100% de energía

In [20]:
shape = Sigma.shape
SVD100 = np.dot(np.dot(U,Sigma),V)
t_final_SVD100 = time()

### Resultados SVD 100% de energía

In [21]:
print(Sigma.shape[0], Sigma.shape[1])

3663 3663


In [22]:
S1, V_cur, D1 = svd(matriz_uspl)

In [23]:
V_cur.size

3706

In [24]:
print(np.linalg.norm(matriz_uspl - SVD100, ord=2),np.linalg.norm(matriz_uspl - CUR, ord=2))

1149.70551989922 10001.031003718312


In [25]:
print(np.linalg.norm(matriz_uspl - SVD100, ord='fro'),np.linalg.norm(matriz_uspl - CUR, ord='fro'))

4730.232971119562 15121.148172147936


In [26]:
print("Error Cuadrático Medio SVD 100%", ECM(SVD100, matriz_uspl))
print("Coeficiente de correlación de Spearman", rho(SVD100, matriz_uspl))
print("Tiempo de procesamiento en segundos SVD 100%", t_final_SVD100 - tiempo_inicial_SVD)

Error Cuadrático Medio SVD 100% 0.9997959061524279
Coeficiente de correlación de Spearman 0.999999999999988
Tiempo de procesamiento en segundos SVD 100% 27420.01447248459


#### Cálculo SVD con retención del 90% de la energía

In [27]:
# Una regla de dedo es retener suficientes valores singulares para representar el 90% de energía

suma_total = 0
dimensions = Sigma.shape[0]

# Calcula el cuadrado de la suma de todas las diagonales
for i in range(dimensions):
    suma_total = suma_total + np.square(Sigma[i,i])	
retencion = suma_total
while dimensions > 0:
    retencion = retencion - np.square(Sigma[dimensions-1,dimensions-1])
    if retencion/suma_total < 0.9: # Retención del 90% de energía
        break
    else:
        U = U[:,:-1:]
        V = V[:-1,:]
        Sigma = Sigma[:,:-1]
        Sigma = Sigma[:-1,:]
        dimensions = dimensions - 1	# reducción de dimensionalidad
t_aux_2 = time()

SVD90 = np.dot(np.dot(U,Sigma),V)
t_final_SVD90 = time()

#### Rango de la matriz SVD90

Verificamos el rango de la matriz SVD90 para tener este número como punto de partida para la selección de columnas y renglones para la descomposición CUR

In [28]:
np.linalg.matrix_rank(SVD90)

883

### Resultados SVD con retención del 90% de la energía

In [29]:
print("Error Cuadrático Medio SVD 90%", ECM(SVD90, matriz_uspl))
print("Coeficiente de correlación de Spearman SVD 90%", rho(SVD90, matriz_uspl))
print("Tiempo de procesamiento en segundos SVD 90%",t_final_SVD90 - t_aux_2 + t_aux_1 - tiempo_inicial_SVD)

Error Cuadrático Medio SVD 90% 0.9692098352347941
Coeficiente de correlación de Spearman SVD 90% 0.9999999999999888
Tiempo de procesamiento en segundos SVD 90% 27393.673482894897


### Pruebas con distintos tamaños de matrices para Descomposición CUR

In [30]:
# Pruebas con matrices de distintos tamaños (3,500 columnas y renglones)
random.seed(5627)

c=3500
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),c)
    
for i in range(c):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=3500
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),r)

for i in range(r):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_3500 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_3500.shape[0]):
    for j in range(CUR_3500.shape[1]):
        if CUR_3500[i,j] > 5:
            CUR_3500[i,j] = 5
        elif CUR_3500[i,j] < 0:
            CUR_3500[i,j] = 0
            
# Pruebas con matrices de distintos tamaños (3,000 columnas y renglones)

c=3000
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),c)
    
for i in range(c):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=3000
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),r)

for i in range(r):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_3000 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_3000.shape[0]):
    for j in range(CUR_3000.shape[1]):
        if CUR_3000[i,j] > 5:
            CUR_3000[i,j] = 5
        elif CUR_3000[i,j] < 0:
            CUR_3000[i,j] = 0
            
#print("\nDescomposición CUR matriz 3000 columnas y renglones")
#print("Error Cuadrático Medio", ECM(CUR_3000, matriz_uspl))
#print("Coeficiente de correlación de Spearman", rho(CUR_3000, matriz_uspl))

# Pruebas con matrices de distintos tamaños (1,200 columnas y renglones)

c=1200
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),c)
    
for i in range(c):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=1200
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),r)

for i in range(r):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_1200 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_1200.shape[0]):
    for j in range(CUR_1200.shape[1]):
        if CUR_1200[i,j] > 5:
            CUR_1200[i,j] = 5
        elif CUR_1200[i,j] < 0:
            CUR_1200[i,j] = 0
            
#print("\nDescomposición CUR matriz 1200 columnas y renglones")
#print("Error Cuadrático Medio", ECM(CUR_1200, matriz_uspl))
#print("Coeficiente de correlación de Spearman", rho(CUR_1200, matriz_uspl))


# Pruebas con matrices de distintos tamaños (883 columnas y renglones)

c=883
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),c)
    
for i in range(c):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=883
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),r)

for i in range(r):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_2 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_2.shape[0]):
    for j in range(CUR_2.shape[1]):
        if CUR_2[i,j] > 5:
            CUR_2[i,j] = 5
        elif CUR_2[i,j] < 0:
            CUR_2[i,j] = 0
            
#print("\nDescomposición CUR matriz 400 columnas y renglones")
#print("Error Cuadrático Medio", ECM(CUR_2, matriz_uspl))
#print("Coeficiente de correlación de Spearman", rho(CUR_2, matriz_uspl))

# Pruebas con matrices de distintos tamaños (50 columnas y renglones)

c=50
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),c)
    
for i in range(c):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=50
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),r)

for i in range(r):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_3 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_3.shape[0]):
    for j in range(CUR_3.shape[1]):
        if CUR_3[i,j] > 5:
            CUR_3[i,j] = 5
        elif CUR_3[i,j] < 0:
            CUR_3[i,j] = 0
            
#print("\nDescomposición CUR matriz 50 columnas y renglones")
#print("Error Cuadrático Medio", ECM(CUR_3, matriz_uspl))
#print("Coeficiente de correlación de Spearman", rho(CUR_3, matriz_uspl))

# Pruebas con matrices de distintos tamaños (10 columnas y renglones)

c=10
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),c)
    
for i in range(c):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=10
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),r)

for i in range(r):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_4 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_4.shape[0]):
    for j in range(CUR_4.shape[1]):
        if CUR_4[i,j] > 5:
            CUR_4[i,j] = 5
        elif CUR_4[i,j] < 0:
            CUR_4[i,j] = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:281: RuntimeWarning: divide by zero encountered in double_scalars


### Comparación Error Cuadrático Medio | Descomposición CUR

In [31]:
print("3500 Columnas y Renglones", ECM(CUR_3500, matriz_uspl))
print("3000 Columnas y Renglones", ECM(CUR_3000, matriz_uspl))
print("1200 Columnas y Renglones", ECM(CUR_1200, matriz_uspl))
print("800 Columnas y Renglones", ECM(CUR, matriz_uspl))
print("883 Columnas y Renglones", ECM(CUR_2, matriz_uspl))
print("50 Columnas y Renglones", ECM(CUR_3, matriz_uspl))
print("10 Columnas y Renglones", ECM(CUR_4, matriz_uspl))

3500 Columnas y Renglones 3.3600187925338165
3000 Columnas y Renglones 3.366418146424702
1200 Columnas y Renglones 3.271967570249869
800 Columnas y Renglones 3.196050201150994
883 Columnas y Renglones 3.254480526323549
50 Columnas y Renglones 2.1327143009952834
10 Columnas y Renglones nan


Definimos todo el procedimiento de la factorización CUR en una función que solo dependa de k, para comparar con distintos valores en una sola corrida.
La función regresa el número k, el ECM, y el tiempo de ejecución en segundos.

In [42]:
def CUR_ECM(k):
    tiempo_1 = time()
    c=k
    col_selec = []
    C = np.zeros([num_usuarios,c])
    col_selec = random.sample(range(num_pelic-1),c)

    for i in range(c):
        x = col_selec[i]
        p = col_norm[x]
        d = np.sqrt(c*p)
        C[:,i] = matriz_df.loc[:,x]/d
        i = i + 1

    r=k
    ren_selec = []
    R = np.zeros([num_pelic,r])
    ren_selec = random.sample(range(num_usuarios-1),r)

    for i in range(r):
        x = ren_selec[i]
        p = row_norm[x]
        d = np.sqrt(r*p)
        R[:,i] = matriz_df.T.loc[:,x]/d
        i = i + 1

    # La matriz U es construida de W por la pseudoinversa de Moore-Penrose

    W = C[ren_selec,:]
    W1, W_cur, W2 = svd(W)
    W_cur = np.diag(W_cur)
    for i in range(W_cur.shape[0]):
        W_cur[i][i] = 1/W_cur[i][i]
    U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
    CUR_k = np.dot(np.dot(C, U), R.T)

    # El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
    # y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

    for i in range(CUR_k.shape[0]):
        for j in range(CUR_k.shape[1]):
            if CUR_k[i,j] > 5:
                CUR_k[i,j] = 5
            elif CUR_k[i,j] < 0:
                CUR_k[i,j] = 0
    tiempo_2 = time()
    
    print(k,ECM(CUR_k, matriz_uspl),tiempo_2-tiempo_1)           
    
    

Se comparan errores ECM con distintos tamaños para la matriz U

In [47]:
for i in range(50,77,2):
    CUR_ECM(i)

50 2.238242241986131 17.923858880996704
52 2.3077780895795748 17.514683485031128
54 2.147225009082285 17.89399003982544
56 2.1094409774408076 17.50899600982666
58 2.050578132543198 17.532268285751343
60 2.111865460832393 17.28779363632202
62 1.9072094561826103 17.15259075164795
64 1.9851923684711479 17.144397974014282
66 2.279210391760093 17.19521975517273
68 2.198465746757217 17.25241994857788
70 2.2340709556513563 18.678383827209473
72 2.1114187018831183 19.589932441711426
74 2.30442416769925 18.031358003616333
76 2.3789282375282546 18.13300895690918


In [44]:
for i in range(50,100,5):
    CUR_ECM(i)

50 2.104861931501364 17.873253107070923
55 2.256003563757642 17.992897272109985
60 2.2477736550193588 18.096657037734985
65 1.954633865693071 17.97298240661621
70 2.1673933605632203 20.128103256225586
75 2.1690981553951696 18.526030778884888
80 2.564285974590722 17.734022617340088
85 2.31214005205749 18.24525737762451
90 2.5400193088569294 17.79748821258545
95 2.338885476034372 19.477964878082275


### Comparación Coeficiente de Correlación de Spearman $\rho$ | Descomposición CUR

In [48]:
print("\n Comparación Coeficiente de correlación de Spearman")
print("3500 Columnas y Renglones", rho(CUR_3500, matriz_uspl))
print("3000 Columnas y Renglones", rho(CUR_3000, matriz_uspl))
print("1200 Columnas y Renglones", rho(CUR_1200, matriz_uspl))
print("800 Columnas y Renglones", rho(CUR, matriz_uspl))
print("400 Columnas y Renglones", rho(CUR_2, matriz_uspl))
print("50 Columnas y Renglones", rho(CUR_3, matriz_uspl))
print("10 Columnas y Renglones", rho(CUR_4, matriz_uspl))


 Comparación Coeficiente de correlación de Spearman
3500 Columnas y Renglones 0.9999999999998648
3000 Columnas y Renglones 0.9999999999998643
1200 Columnas y Renglones 0.9999999999998718
800 Columnas y Renglones 0.9999999999998777
400 Columnas y Renglones 0.9999999999998732
50 Columnas y Renglones 0.9999999999999455
10 Columnas y Renglones nan
